In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Suyash\\Desktop\\End_to_End_projects\\Text_Summarizer\\research'

In [3]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Suyash\\Desktop\\End_to_End_projects\\Text_Summarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    warmup_steps: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int

In [7]:
from textsummerizer.constants import *
from textsummerizer.utils.common import read_yaml, create_directory

In [8]:
# Define the configuration
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
          
          self.config = read_yaml(config_filepath)
          self.params = read_yaml(params_filepath)

          create_directory([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainingConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directory([config.root_dir])

        model_training_config = ModelTrainingConfig(
             root_dir=config.root_dir,
             data_path=config.data_path,
             model_ckpt=config.model_ckpt,
             num_train_epochs=params.num_train_epochs,
             warmup_steps=params.warmup_steps,
             per_device_train_batch_size=params.per_device_train_batch_size,
             per_device_eval_batch_size=params.per_device_eval_batch_size,
             weight_decay=params.weight_decay,
             logging_steps=params.logging_steps,
             evaluation_strategy=params.evaluation_strategy,
             eval_steps=params.eval_steps,
             save_steps=params.save_steps,
             gradient_accumulation_steps=params.gradient_accumulation_steps,
        )

        return model_training_config
        

In [9]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, load_from_disk
import torch

c:\Users\Suyash\anaconda3\envs\textSummarizer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Define the model trainer
class ModelTrainer:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model_ckpt = "google/pegasus-cnn_dailymail"
        tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # Load the dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            warmup_steps=self.config.warmup_steps,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
        )

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset = dataset_samsum_pt["test"],
                  eval_dataset = dataset_samsum_pt["validation"])
        
        trainer.train()

        # Save the model and tokenizer
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus_samsum_model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "pegasus_samsum_tokenizer"))


In [11]:
# building pipeline

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)    
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-02-03 23:48:47,377: INFO: common: Yaml file: config\config.yaml loaded successfully]
[2024-02-03 23:48:47,389: INFO: common: Yaml file: params.yaml loaded successfully]
[2024-02-03 23:48:47,394: INFO: common: directory created at artifacts]
[2024-02-03 23:48:47,398: INFO: common: directory created at artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/51 [00:00<?, ?it/s]
KeyboardInterrupt

